# K-means implementation

In [ ]:
!pip install matplotlib
!pip install numpy
!pip install pandas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_table("cluster.dat", sep=" ", header=None)
X = dataset.iloc[:, 0:1].values
Y = dataset.iloc[:, 1:2].values

plt.scatter(X, Y, label = "Point (X;Y)", color = 'k')
plt.xlabel('X')
plt.ylabel('Y')
plt.title("Points dataset for finding centroid coordinates")
plt.show()

In [24]:
def inicializaCentroides(k, data):
    centroides = []
    for i in range(0,k):
        centroides.append(data.loc[[i]])
    return centroides

def calculaDistancia(ponto1, ponto2):
    print(ponto1, ponto2)

def encontraCentroideMaisProximo(ponto, centroides, k):
    ponto = ponto.to_numpy()
    for centroide in centroides:
        # calcula distancia pra cada centroide
        print("oi", centroide)
        calculaDistancia(ponto, centroide)
    #print(centroides)

def inicializaClusters(centroides, data, k):
    clusters = []
    for i in range(0,k):
        clusters.append([])
    for index, i in data.iterrows():
        #calcula media e distancia
        encontraCentroideMaisProximo(i, centroides, k)
        #atribui a um cluster

def kmeans(k, data):
    # Passo 1: Fornece valores iniciais para os centroides e agrupa
    #         o restante dos dados em um centroide
    centroides = inicializaCentroides(k, data)
    inicializaClusters(centroides, data, k)
    
    # Passo 2: Gera uma matriz de distancia entre cada ponto e 
    #         os centroides (N x k)
    # Passo 3: Coloca cada ponto nas classes que está mais próximo
    # Passo 4: Calcula os novos centroides para cada classe com a média
    # Passo 5: Repete o passo 2 em diante até convergir
    
kmeans(3, dataset)

oi         0      1
0  1555.0  28.65
[1555.     28.65]         0      1
0  1555.0  28.65
oi         0      1
1  1490.0  27.55
[1555.     28.65]         0      1
1  1490.0  27.55
oi         0      1
2  1445.0  28.35
[1555.     28.65]         0      1
2  1445.0  28.35
oi         0      1
0  1555.0  28.65
[1490.     27.55]         0      1
0  1555.0  28.65
oi         0      1
1  1490.0  27.55
[1490.     27.55]         0      1
1  1490.0  27.55
oi         0      1
2  1445.0  28.35
[1490.     27.55]         0      1
2  1445.0  28.35
oi         0      1
0  1555.0  28.65
[1445.     28.35]         0      1
0  1555.0  28.65
oi         0      1
1  1490.0  27.55
[1445.     28.35]         0      1
1  1490.0  27.55
oi         0      1
2  1445.0  28.35
[1445.     28.35]         0      1
2  1445.0  28.35
oi         0      1
0  1555.0  28.65
[1415.    28.8]         0      1
0  1555.0  28.65
oi         0      1
1  1490.0  27.55
[1415.    28.8]         0      1
1  1490.0  27.55
oi         0      1
2  14